In [1]:
# TO DO: implement beta experiments, on 50-50 probability and Klaassen probabilities
# analyze different methods

# REWRITE THE PROBABILITY COMPUTATIONS??
# test each method on a subset of matches, via tourney_id

# prior experiments: test priors with beta experiments

In [2]:
import sys
sys.path.insert(0,'/Users/jacobgollub/Desktop/college (current)/research/pbp_explorations/scripts/sackmann')
import tennisGameProbability,tennisMatchProbability,tennisSetProbability,tennisTiebreakProbability
from tennisMatchProbability import matchProb
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss

df = pd.read_csv('../my_data/feature_df_pbp3_8_22.csv')
del df['Unnamed: 0']

# tour-level averages for serve and return points won
f_av = 0.646775
g_av = 1 - f_av

df['tourney_stats'] = [f_av if stat==0 else stat for stat in df['tourney_stats']]
df['p0_s_kls'] = df['tourney_stats'] + (df['p0_s_pct']-f_av) - (df['p1_r_pct']-g_av)
df['p1_s_kls'] = df['tourney_stats'] + (df['p1_s_pct']-f_av) - (df['p0_r_pct']-g_av)
df['p0_s_kls_JS'] = df['tourney_stats'] + (df['p0_s_pct_JS']-f_av) - (df['p1_r_pct_JS']-g_av)
df['p1_s_kls_JS'] = df['tourney_stats'] + (df['p1_s_pct_JS']-f_av) - (df['p0_r_pct_JS']-g_av)

cols = [u'match_id',u'sets_0',u'sets_1', u'games_0', u'games_1', u'points_0', u'points_1','p0_s_pct','p0_r_pct',
        'p1_s_pct','p1_r_pct','p0_s_kls','p1_s_kls','p0_s_kls_JS','p1_s_kls_JS','tourney_stats','best_of',u'winner']
df = df[cols]
df['winner'] = 1 - df['winner']

In [35]:
df.shape

(1231122, 16)

In [12]:
# when we don't use a prior, or assign weights to evidence, we get serving percentages like this one...
# even though Tursunov won the match easily, you want to increase the serving percentage of p0
# since Tursunov's 25/32 return percentage was way the hell too high (it's these kinds of things that hurt the
# cross entropy score...); match_id: 6439
df['match_id'][760117],df['p0_s_kls'][760117]

(6439.0, 0.15158511132899999)

In [29]:
# before: 4500 with negative probabilities
# df[df['klaassen_prob']<0]

In [5]:
# TO DO: implement Klaassen's method, based on serve and return stats
# then, test a grid search of averaging historical with overall player percentages and see if any split (50-50,70-30,etc)
# does best
# then, run the beta experiments outlined in pbp paper, also running grid search on alpha parameter

# The numbers I used for men are: 63% in Australia, 61% at the French, 66% at Wimbledon, and 64% at the U.S. Open

In [ ]:
import time
start = time.clock()

#df['sackmann_prob'] = np.zeros(len(df))
df['klaassen_prob'] = np.zeros(len(df))
df['klaassen_prob_JS'] = np.zeros(len(df))
for i,row in df.iterrows():
#     df['sackmann_prob'][i] = matchProb(f_av,g_av,mv=int(row['sets_0']),mw=int(row['sets_1']),sv=int(row['games_0']), \
#                             sw=int(row['games_1']),gv=int(row['points_0']),gw=int(row['points_1']))
    df['klaassen_prob'][i] = matchProb(row['p0_s_kls'],1-row['p1_s_kls'],mv=int(row['sets_0']),mw=int(row['sets_1']), \
                            sv=int(row['games_0']),sw=int(row['games_1']),gv=int(row['points_0']),gw=int(row['points_1']))
    df['klaassen_prob_JS'][i] = matchProb(row['p0_s_kls_JS'],1-row['p1_s_kls_JS'],mv=int(row['sets_0']),mw=int(row['sets_1']), \
                            sv=int(row['games_0']),sw=int(row['games_1']),gv=int(row['points_0']),gw=int(row['points_1']))


#     print int(row['sets_0']),int(row['sets_1']),int(row['games_0']),int(row['games_1']), \
#                   int(row['points_0']),int(row['points_1'])
    if i%300000==0:
        print i

print time.clock()-start

0
300000

In [27]:
#set(df[df['klaassen_prob']!=df['klaassen_prob']]['match_id'])

In [ ]:
from sklearn.metrics import log_loss

k_df = df[df['klaassen_prob']==df['klaassen_prob']]
n = len(k_df)
k_score = log_loss(k_df['winner'][:n],k_df['klaassen_prob'][:n],labels=[0,1])
print 'klaassen score: ',k_score

k_df = df[df['klaassen_prob_JS']==df['klaassen_prob_JS']]
n = len(k_df)
k_score = log_loss(k_df['winner'][:n],k_df['klaassen_prob_JS'][:n],labels=[0,1])
print 'klaassen score JS: ',k_score

In [18]:
from sklearn.metrics import log_loss
# evaluate log_loss of these probabilities (log_loss(y_true,y_pred))
# we get .54 log_loss with no information about the players... (around .48 was best with most recent features)

# n=len(df)
# score = log_loss(df['winner'][:n],df['sackmann_prob'][:n],labels=[0,1])
# print 'sackmann score: ',score

k_df = df[df['klaassen_prob']==df['klaassen_prob']]
k_score = log_loss(k_df['winner'][:n],k_df['klaassen_prob'][:n],labels=[0,1])
print 'klaassen score: ',k_score
#df.to_csv('../my_data/feature_df_pbp3_7_30_s_prob.csv')

sackmann score:  0.538905529928
klaassen score:  0.591980500589


In [53]:
.4*.6

0.24

In [54]:
arr = [0,0,1,1,1]
np.var(arr)

0.24000000000000005

In [ ]:
# import pandas as pd
# import numpy as np

# # storing coefficients for serve and return shrinkage in dicts
# df_0 = pd.read_csv('../my_data/elo_pbp_with_surface_8_14.csv')

# # calculate B_i coefficients for each player in terms of service points; since serve and return points are approximately
# # equal, we use the same coefficient for both
# s_history = np.concatenate([df_0['p0_52_swon']/df_0['p0_52_svpt'],df_0['p1_52_swon']/df_0['p1_52_svpt']],axis=0)
# p_hat = np.sum([df_0['p0_52_swon'],df_0['p1_52_swon']])/np.sum([df_0['p0_52_svpt'],df_0['p1_52_svpt']])
# sigma2_i = np.nan_to_num(p_hat*(1-p_hat)/np.concatenate([df_0['p0_52_svpt'],df_0['p1_52_svpt']]))
# tau2_hat = np.nanvar(s_history)
# B_i = sigma2_i/(tau2_hat+sigma2_i)
# df_0['B_i0_sv'],df_0['B_i1_sv'] = B_i[::2],B_i[1::2]

# n = len(s_history)/2
# s_history[s_history!=s_history] = p_hat
# df_0['p0_s_pct_JS'] = s_history[:n]+df_0['B_i0_sv']*(p_hat-s_history[:n])
# df_0['p1_s_pct_JS'] = s_history[n:]+df_0['B_i1_sv']*(p_hat-s_history[n:])

# # repeat for return averages (slightly different tau^2 value)
# r_history = np.concatenate([df_0['p0_52_rwon']/df_0['p0_52_rpt'],df_0['p1_52_rwon']/df_0['p1_52_rpt']],axis=0)
# sigma2_i = np.nan_to_num((1-p_hat)*p_hat/np.concatenate([df_0['p0_52_rpt'],df_0['p1_52_rpt']]))
# tau2_hat = np.nanvar(r_history)
# B_i = sigma2_i/(tau2_hat+sigma2_i)
# df_0['B_i0_r'],df_0['B_i1_r'] = B_i[::2],B_i[1::2]

# r_history[r_history!=r_history] = p_hat
# df_0['p0_r_pct_JS'] = r_history[:n]+df_0['B_i0_r']*(p_hat-r_history[:n])
# df_0['p1_r_pct_JS'] = r_history[n:]+df_0['B_i1_r']*(p_hat-r_history[n:])